In [9]:
import pandas as pd
import os

df_customer = pd.read_csv("AnnualCustomerData_01.18.2026.csv")
df_daily = pd.read_csv("daily_data_noram.csv")
df_monthly = pd.read_csv("monthly_data_noram.csv")


df_daily["report_date"] = pd.to_datetime(df_daily["report_date"])
df_monthly["report_date"] = pd.to_datetime(df_monthly["report_date"])
df_customer["fw1__Billing_Period_Start__c"] = pd.to_datetime(df_customer["fw1__Billing_Period_Start__c"])
df_customer["fw1__Billing_Period_End__c"] = pd.to_datetime(df_customer["fw1__Billing_Period_End__c"])

In [20]:
# individual period compare monthly vs. daily counts

start_date = "2024-04-01"
end_date   = "2024-04-30"
sf_account = "0011200001DX2m0AAD"

def filter_by_account(df, start_date, end_date, sf_account):
    return df[
        (df["report_date"].between(start_date, end_date)) &
        (df["salesforceaccountid"] == sf_account)
    ]

daily_filtered = filter_by_account(df_daily, start_date, end_date, sf_account)
monthly_filtered = filter_by_account(df_monthly, start_date, end_date, sf_account)

total_daily_invoices = daily_filtered["invoice_cnt"].sum()
total_monthly_invoices = monthly_filtered["invoice_cnt"].sum()

print("Total daily invoices:", total_daily_invoices)
print("Total monthly invoices:", total_monthly_invoices)

Total daily invoices: 310
Total monthly invoices: 310


In [3]:
df_customer["First_Month_Usage"] = 0
df_customer["Mid_Month_Usage"] = 0
df_customer["Last_Month_Usage"] = 0
df_customer["Total_Usage"] = 0

In [31]:
for row in df_customer.itertuples(index=True):
    idx = row.Index
    acct = row.fw1__Account__c
    start = pd.to_datetime(row.fw1__Billing_Period_Start__c)
    end   = pd.to_datetime(row.fw1__Billing_Period_End__c)

    # Calculate first month usage with daily values
    first_month_start = start
    first_month_end = (start + pd.offsets.MonthEnd(0))

    daily_first = df_daily[(df_daily["salesforceaccountid"] == acct) &
        (df_daily["report_date"].between(first_month_start, first_month_end))]["invoice_cnt"].sum()

     # Calculate mid-month usage with monthly values
    mid_start = (start + pd.offsets.MonthBegin(1))
    mid_end   = (end - pd.offsets.MonthBegin(1) - pd.Timedelta(days=1))

    monthly_mid = df_monthly[(df_monthly["salesforceaccountid"] == acct) &
            (df_monthly["report_date"].between(mid_start, mid_end))]["invoice_cnt"].sum()

    # Calculate last month usage with daily values
    last_month_start = end.replace(day=1)
    last_month_end = end

    daily_last = df_daily[
        (df_daily["salesforceaccountid"] == acct) &
        (df_daily["report_date"].between(last_month_start, last_month_end))]["invoice_cnt"].sum()

    # ---- Write results ----
    df_customer.at[idx, "First_Month_Usage"] = daily_first
    df_customer.at[idx, "Mid_Month_Usage"]   = monthly_mid
    df_customer.at[idx, "Last_Month_Usage"]  = daily_last
    df_customer.at[idx, "Total_Usage"]       = daily_first + monthly_mid + daily_last

    

In [32]:
df_customer

,Id,fw1__Status__c,fw1__Account__c,fw1__Account__r.Name,fw1__Account__r.Sales_Organization__c,fw1__Account__r.CurrencyIsoCode,fw1__Recurring_Invoice__r.RecordType.Name,fw1__Recurring_Invoice__r.Rollup_Invoice_Tier__c,fw1__Recurring_Invoice__r.fw1__Opportunity__r.Cost_Invoice__c,fw1__Billing_Period_Start__c,fw1__Billing_Period_End__c,First_Month_Usage,Mid_Month_Usage,Last_Month_Usage,Total_Usage
0,a52OL000007TjM6YAK,Paid,0011200001DX2m0AAD,Prostar Well Service Inc.,CA00,CAD,BWAP,200,2.610000,2024-04-01,2025-03-31,310.0,3298.0,418.0,4026.0
1,a52OL00000MzWZOYA3,Paid,0011200001DX2m0AAD,Prostar Well Service Inc.,CA00,CAD,BWAP,200,2.610000,2024-04-01,2025-03-31,310.0,3298.0,418.0,4026.0
2,a52OL00000GwmswYAB,Paid,0011200001F61OFAAZ,Boon Energy Services,CA00,CAD,BWAP,200,2.610000,2024-10-01,2025-09-30,232.0,2240.0,282.0,2754.0
3,a52OL000004TDEpYAO,Paid,0011200001FVPO1AAP,TA Management,CA00,CAD,BWAP,300,1.793333,2024-02-01,2025-01-31,309.0,3524.0,394.0,4227.0
4,a52OL000004TDEbYAO,Paid,0011200001FwddVAAR,Resource Environmental Inc.,US00,USD,BWAP,300,1.793333,2024-02-01,2025-01-31,249.0,3402.0,402.0,4053.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,a52OL00000QwRwwYAF,Paid,001OL00000HjEcvYAF,Empacta Development Consultants,CA00,CAD,BWAP,50,2.800000,2024-11-01,2025-10-31,28.0,520.0,66.0,614.0
541,a52OL00000JHZZWYA5,Paid,001OL00000I95kVYAR,"AAFCPAs, Inc.",US00,USD,BWAP,100,1.890000,2024-11-26,2025-11-25,0.0,1063.0,99.0,1162.0
542,a52OL00000KM439YAD,Paid,001OL00000JUsT3YAL,Kelly Partners,US00,USD,BWAP,100,1.890000,2024-11-21,2025-11-20,0.0,664.0,48.0,712.0
543,a52OL00000MFIzGYAX,Paid,001OL00000Jmi5RYAR,First Peak Construction,US00,USD,BWAP,100,1.890000,2024-11-14,2025-11-13,28.0,598.0,36.0,662.0
